In [15]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.messages import HumanMessage
from langchain_core.prompts import *
from langchain.tools import tool
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from tenacity import retry, stop_after_attempt, wait_exponential

@tool
def trident_cve_tool(cve_id: str):
    """
    This is a CVE tool to call trident CVE information.
    It should be used to fetch general information of a specific cve (format like CVE-XXX-XXX).
    The general information from Trident API includes:
    cve id, name, description, update date, publish date and some other public available information.
    """
    return str("Trident CVE data")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["cve_id"],
                template="You are a helpful assistant to explain a certain CVE {cve_id} in detail",
            )
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["question"],
                template="Please answer this question: {question}",
            )
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
tools = [trident_cve_tool]

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def create_llm():
    return AzureChatOpenAI(
        temperature=0.0,
        azure_endpoint="http://localhost:8000",
        openai_api_key="bedrock",
        openai_api_version="2024-06-01",
        deployment_name="anthropic.claude-3-5-sonnet-20240620-v1:0",    
        # deployment_name="gpt-4o_2024-05-13",
        openai_api_type="azure",
    )




In [16]:
llm = create_llm()
llm.temperature = 0.0
agent = create_tool_calling_agent(llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
content = agent_executor.invoke(
    {"question": "What is CVE-2020-17087?", "cve_id": "CVE-2020-17087"}
)
print(content)




> Entering new AgentExecutor chain...

Invoking: `trident_cve_tool` with `{'cve_id': 'CVE-2020-17087'}`


Trident CVE data

CVE-2020-17087 is a vulnerability in the Linux kernel's VFIO device driver that could allow a privileged user to escalate privileges or cause a denial of service on the system.

The VFIO (Virtual Function I/O) driver allows safe and efficient device access from userspace on Linux hosts. This vulnerability is a use-after-free issue that could occur when removing a VFIO device. An unprivileged user with access to a VFIO device could potentially exploit this to gain elevated privileges on the system.

Some key details about CVE-2020-17087:

- Published: 2020-08-12
- CVSS Score: 7.8 (High severity)
- Affects: Linux kernel versions 5.4 through 5.8.5
- Type: Use-after-free vulnerability in vfio_unbound_driver()
- Impact: Privilege escalation, denial of service

The vulnerability was discovered by Cedric Hombracht and reported to Linux kernel security teams. It was pat